In [3]:
import numpy
import matplotlib.pyplot as plt 
%matplotlib inline
plt.rcParams["figure.figsize"]=[5,5]

In [ ]:
#Convoluciones 
#Es una operacion entre matrices donde una matriz pequeña (kernel o filtro) se aplica de forma iterativa a los distintos trozos de una matriz mas grande (una imagen)

#Se va a ver de forma practica el ejemplo de convolucion visto en la teoria. para ello creamos una imagen con dos trozos verticales, una blanco y otro negro

In [ ]:
img_size=64
img_borde_vertical=numpy.zeros((img_size, img_size))

img_borde_vertical[:,:int(img_size/2)]=1

plt.imshow(img_borde_vertical, cmap="gray")
plt.title("Imagen original")

In [ ]:
#Ahora vamos a aplicarle un filtro de deteccion de bordes verticales (filtro sobel vertical) y ver el output. Para ello usaremos la funcion de scipy convolve2d que realiza la convolucion

In [ ]:
from scipy import signal

filtro_sobel_vertical=numpy.array([[-1,0,1],[-1,0,1],[-1,0,1]])

output_convolucion=signal.convolve2d(img_borde_vertical, filtro_sobel_vertical, mode="valid")
plt.imshow(numpy.absolute(output_convolucion), cmap="gray")
plt.title("Output de la convolucion con filtro vertical")

In [ ]:
#Ahora se va a cargar una imagen 

In [ ]:
from scipy import misc
img_original=misc.ascent()

plt.imshow(img_original, cmap="gray")
plt.title("imagen original")

In [ ]:
filtro_sobel_horizontal=numpy.array([[-1,-1,-1], [0,0,0], [1,1,1]])
output_convolucion=signal.convolve2d(img_original, filtro_sobel_horizontal)
plt.imshow(output_convolucion, cmap="gray")
plt.title("Output de la convolucion con filtro horinzontal")

In [ ]:
#Redes Neuronales de convolucion

In [ ]:
#Cargamos los datos
#En este caso vamos a usar un dataset nuevo, el Fashion MNIST, que es un dataset mas moderno que el dataset de digitos MNIST, que hoy en dia se considera un dataset demasiado "facil", y que puede ser resuelto con modelos que no estan relacionados con la vision artificial (como los SVMs)

#En concreto el dataset Fashion MNIST consiste en 60,000 fotos (28,28pixeles como el dataset de digitos) de prendas de ropa y su objetivo es clasificar las imagenes respecto al tipo de ropa mostrado (10 tipos distintos de articulos)

In [4]:
from keras.datasets import fashion_mnist
(x_train, y_train), (x_test, y_test)=fashion_mnist.load_data()

In [5]:
x_train.shape

(60000, 28, 28)

In [6]:
#Se puede usar numpy.bincount para contar cuantas observaciones hay de cada clase de prenda

In [4]:
numpy.bincount(y_train)

array([6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000],
      dtype=int64)

In [ ]:
#Convertimos la variable objetivo a vectores one hot

In [7]:
from keras.utils.np_utils import to_categorical

y_train_one_hot=to_categorical(y_train)
y_test_one_hot=to_categorical(y_test)

In [5]:
plt.rcParams["figure.figsize"]=[4,4]

In [ ]:
from ipywidgets import interact, IntSlider

@interact(i=IntSlider(min=0, max=100, step=1, value=1))
def dibujar_imagen(i):
    plt.imshow(x_train[i], cmap="gray")
    plt.title("clase de prenda: {}".format(y_train[i]))

In [ ]:
#En primer lugar se va a usar una red densa y la vamos a evaluar

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [7]:
#Aplanamos las imagenes

In [6]:
x_train_plano=x_train.reshape(x_train.shape[0], 28*28)
x_test_plano=x_test.reshape(x_test.shape[0], 28*28)

In [7]:
x_train_plano.shape

(60000, 784)

In [9]:
x_train[0].shape

(28, 28)

In [16]:
modelo_denso=Sequential()
modelo_denso.add(Dense(128, activation="relu", input_shape=(784,)))
modelo_denso.add(Dropout(0.2))
modelo_denso.add(Dense(256, activation="relu"))
modelo_denso.add(Dropout(0.2))
modelo_denso.add(Dense(128, activation="relu"))
modelo_denso.add(Dropout(0.2))
modelo_denso.add(Dense(numpy.unique(y_train).shape[0], activation="softmax"))

modelo_denso.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

modelo_denso.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 128)               100480    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               33024     
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)               

In [17]:
numpy.unique(y_train).shape[0]

10

In [18]:
modelo_denso.fit(x_train_plano, y_train_one_hot, epochs=30, batch_size=500, verbose=0)
modelo_denso.evaluate(x_test_plano, y_test_one_hot)

313/313 [==============================] - 1s 3ms/step - loss: 0.4033 - accuracy: 0.8556


[0.4033378064632416, 0.8555999994277954]

In [ ]:
#[0.4033378064632416, 0.8555999994277954]

In [ ]:
#Ahora vamos a usar una red de convolucion (CNN)

In [19]:
img_rows, img_cols=28, 28
x_train=x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test=x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape=(img_rows, img_cols)

In [10]:
x_train.shape

(60000, 28, 28)

In [20]:
from keras.layers import Conv2D, Flatten, MaxPooling2D

batch_size=256
num_classes=10
epochs=10

input_shape=(28,28,1)

modelo_cnn=Sequential()
modelo_cnn.add(Conv2D(32, kernel_size=(3,3), activation="relu", input_shape=input_shape))
modelo_cnn.add(MaxPooling2D(pool_size=(2,2)))
modelo_cnn.add(Dropout(0.3))
modelo_cnn.add(Flatten())
modelo_cnn.add(Dense(32, activation="relu"))
modelo_cnn.add(Dropout(0.5))
modelo_cnn.add(Dense(num_classes, activation="softmax"))
modelo_cnn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 32)                173088    
_________________________________________________________________
dropout_6 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)               

In [21]:
modelo_cnn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
modelo_cnn.fit(x_train, y_train_one_hot, epochs=5, batch_size=1000, verbose=1)

Epoch 1/5
60/60 [==============================] - 19s 300ms/step - loss: 4.9375 - accuracy: 0.1539
Epoch 2/5
60/60 [==============================] - 18s 304ms/step - loss: 2.1001 - accuracy: 0.1876
Epoch 3/5
60/60 [==============================] - 18s 304ms/step - loss: 2.0567 - accuracy: 0.1956
Epoch 4/5
60/60 [==============================] - 18s 302ms/step - loss: 1.9927 - accuracy: 0.2077
Epoch 5/5
60/60 [==============================] - 18s 305ms/step - loss: 1.9558 - accuracy: 0.2131


In [22]:
modelo_cnn.evaluate(x_tes, y_test_one_hot, verbose=1)

313/313 [==============================] - 2s 5ms/step - loss: 1.5826 - accuracy: 0.3926


[1.582605004310608, 0.39259999990463257]

In [23]:
#[1.582605004310608, 0.39259999990463257]

In [27]:
from keras.layers import BatchNormalization
from keras.callbacks import EarlyStopping

batch_size=128
num_classes=10
epochs=2
filter_pixel=3
noise=1
droprate=0.35

model=Sequential()
model.add(Conv2D(64, kernel_size=(filter_pixel, filter_pixel), padding="same", activation="relu", input_shape=input_shape))
model.add(BatchNormalization())
model.add(Dropout(droprate))

model.add(Conv2D(64, kernel_size=(filter_pixel, filter_pixel), activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Dropout(droprate))

model.add(Conv2D(64, kernel_size=(filter_pixel, filter_pixel), activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Dropout(droprate))

model.add(Flatten())
model.add(Dense(500, use_bias=False, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(droprate))

model.add(Dense(10,activation="softmax"))

model.summary()


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 28, 28, 64)        640       
_________________________________________________________________
batch_normalization_5 (Batch (None, 28, 28, 64)        256       
_________________________________________________________________
dropout_12 (Dropout)         (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 28, 28, 64)        36928     
_________________________________________________________________
batch_normalization_6 (Batch (None, 28, 28, 64)        256       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 14, 14, 64)       

In [29]:
callbacks=[
    EarlyStopping(
        monitor="val_acc",
        patience=1,
        mode="max",
        verbose=1
    )
]

model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

history=model.fit(x_train, y_train_one_hot, batch_size=batch_size,epochs=2)

Epoch 1/2
191/469 [===========>..................] - ETA: 3:20 - loss: 0.8838 - accuracy: 0.7214

KeyboardInterrupt: 

In [32]:
model.evaluate(x_test, y_test_one_hot, verbose=1)

313/313 [==============================] - 14s 43ms/step - loss: 0.4509 - accuracy: 0.8357


[0.45087411999702454, 0.8356999754905701]

In [33]:
#[0.45087411999702454, 0.8356999754905701]

In [34]:
import keras.backend as K
#https://github.com/philipperemy/keras-visualize-activations/blob/master/read_activations.py
def get_activations(model, model_inputs, print_shape_only=False, layer_name=None):
    print('----- activations -----')
    activations = []
    inp = model.input

    model_multi_inputs_cond = True
    if not isinstance(inp, list):
        # only one input! let's wrap it in a list.
        inp = [inp]
        model_multi_inputs_cond = False

    outputs = [layer.output for layer in model.layers if
               layer.name == layer_name or layer_name is None]  # all layer outputs

    funcs = [K.function(inp + [K.learning_phase()], [out]) for out in outputs]  # evaluation functions

    if model_multi_inputs_cond:
        list_inputs = []
        list_inputs.extend(model_inputs)
        list_inputs.append(0.)
    else:
        list_inputs = [model_inputs, 0.]

    layer_outputs = [func(list_inputs)[0] for func in funcs]
    for layer_activations in layer_outputs:
        activations.append(layer_activations)
        if print_shape_only:
            print(layer_activations.shape)
        else:
            print(layer_activations)
    return activations

def display_activations(activation_maps):
    batch_size = activation_maps[0].shape[0]
    assert batch_size == 1, 'One image at a time to visualize.'
    for i, activation_map in enumerate(activation_maps):
        print('Displaying activation map {}'.format(i))
        shape = activation_map.shape
        if len(shape) == 4:
            print(shape)
            activations = np.hstack(np.transpose(activation_map[0], (2, 0, 1)))
        elif len(shape) == 2:
            # try to make it square as much as possible. we can skip some activations.
            activations = activation_map[0]
            num_activations = len(activations)
            if num_activations > 1024:  # too hard to display it on the screen.
                square_param = int(np.floor(np.sqrt(num_activations)))
                activations = activations[0: square_param * square_param]
                activations = np.reshape(activations, (square_param, square_param))
            else:
                activations = np.expand_dims(activations, axis=0)
        else:
            raise Exception('len(shape) = 3 has not been implemented.')
        plt.imshow(activations, interpolation='None', cmap='jet')
        plt.show()